In [ ]:
import requests
import csv

In [ ]:
headers=["match_id", "date", "team", "runs", "overs", "batting_first", "player", "id", "role", "impact_sub", "replaced", "over", "innings"]

In [ ]:
def queryCricinfo(url):
  resp = requests.get(url, headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'})

  return resp.json()

In [ ]:
def getMatch(id, seriesId):
  resp = queryCricinfo(f"https://hs-consumer-api.espncricinfo.com/v1/pages/match/scorecard?lang=en&seriesId={seriesId}&matchId={id}")

  if resp["match"]["stage"] == "SCHEDULED":
    return

  if "content" not in resp:
    return

  if "matchPlayers" not in resp["content"] or resp["content"]["matchPlayers"] == None:
    return

  runs = {}
  overs = {}
  teamPlayers = resp["content"]["matchPlayers"]["teamPlayers"]
  id = resp["match"]["objectId"]
  date = resp["match"]["startDate"]
  teamBattingFirst = ""

  for inn in resp["content"]["innings"]:
    runs[inn["team"]["longName"]] = inn["runs"]
    overs[inn["team"]["longName"]] = inn["overs"]

    if inn["inningNumber"] == 1:
      teamBattingFirst = inn["team"]["longName"]

  replacement = resp["match"]["replacementPlayers"]

  subs = {}
  if replacement is not None:
    for rep in replacement:
      team = rep["team"]["longName"]
      sub = rep["player"]["longName"]
      subID = rep["player"]["objectId"]
      replaced = rep["replacingPlayer"]["longName"]
      replacedID = rep["replacingPlayer"]["objectId"]
      over= rep["over"]
      inn = rep["inning"]

      subs[team]= {
          "sub": sub,
          "subID": subID,
          "replaced": replaced,
          "replacedID": replacedID,
          "over": over,
          "inn": inn
      }

  data = []
  for team in teamPlayers:
    name = team["team"]["longName"]
    battingFirst = name == teamBattingFirst
    for player in team["players"]:
      playerName = player["player"]["longName"]
      playerID = player["player"]["objectId"]
      role = "/".join(player["player"]["playingRoles"])

      imp = False
      rep = False
      inn = ""
      over = ""

      if name in subs:
        sub = subs[name]

        if playerID == sub["subID"]:
          imp = True
          inn = sub["inn"]
          over = sub["over"]

        if playerID == sub["replacedID"]:
          rep = True
          inn = sub["inn"]
          over = sub["over"]

      runsScored = runs[name] if name in runs else ""
      oversBowled = overs[name] if name in overs else ""
      data.append([id,date,name, runsScored, oversBowled, battingFirst,playerName, playerID, role,imp,rep,over,inn])

  return data

In [ ]:
def getSeries(id):
  resp = queryCricinfo(f"https://hs-consumer-api.espncricinfo.com/v1/pages/series/schedule?lang=en&seriesId={id}")
  data = []
  print("No. of matches: ", len(resp["content"]["matches"]))
  nones = 0
  noDataMatches = []
  for mat in resp["content"]["matches"]:

    matchData= getMatch(mat["objectId"], id)
    if matchData is None:
      nones += 1
      noDataMatches.append(mat["slug"])
    else:
      data.extend(matchData)

  print(f"Match details not found for {nones}  matches.")
  print(", ".join(noDataMatches))

  return data

In [ ]:
series = ["1410320", "1345038", "1298423"]
data = []
for s in series:
  data.extend(getSeries(s))

No. of matches:  74
Match details not found for 45  matches.
royal-challengers-bengaluru-vs-sunrisers-hyderabad-30th-match, kolkata-knight-riders-vs-rajasthan-royals-31st-match, gujarat-titans-vs-delhi-capitals-32nd-match, punjab-kings-vs-mumbai-indians-33rd-match, lucknow-super-giants-vs-chennai-super-kings-34th-match, delhi-capitals-vs-sunrisers-hyderabad-35th-match, kolkata-knight-riders-vs-royal-challengers-bengaluru-36th-match, punjab-kings-vs-gujarat-titans-37th-match, rajasthan-royals-vs-mumbai-indians-38th-match, chennai-super-kings-vs-lucknow-super-giants-39th-match, delhi-capitals-vs-gujarat-titans-40th-match, sunrisers-hyderabad-vs-royal-challengers-bengaluru-41st-match, kolkata-knight-riders-vs-punjab-kings-42nd-match, delhi-capitals-vs-mumbai-indians-43rd-match, lucknow-super-giants-vs-rajasthan-royals-44th-match, gujarat-titans-vs-royal-challengers-bengaluru-45th-match, chennai-super-kings-vs-sunrisers-hyderabad-46th-match, kolkata-knight-riders-vs-delhi-capitals-47th-mat

In [ ]:
csvRows = []
csvRows.append(headers)
csvRows.extend(data)

In [ ]:
with open('impact_sub_data.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for row in csvRows:
      if row is not None:
        writer.writerow(row)